# Aerial Cactus Identification

# Load Packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import zipfile 

# Load DataFrame

In [ ]:
train = pd.read_csv("../input/aerial-cactus-identification/train.csv", dtype=str)
print(train.shape)

In [ ]:
train.head(10)

# Extract Images

In [ ]:
zip_ref = zipfile.ZipFile('/kaggle/input/aerial-cactus-identification/train.zip')
zip_ref.extractall()

In [ ]:
train_path = "train/"
print('Training Images:', len(os.listdir(train_path)))

In [ ]:
print('Training Images:', len(os.listdir('train/')))

for i in range(10):
  img = plt.imread('train/' + train.id[i])
  print('Images shape', img.shape)

# View Sample of Images

In [ ]:
sample = train.sample(n=16).reset_index()

plt.figure(figsize=(8,8))

for i, row in sample.iterrows():

    img = mpimg.imread(f'train/{row.id}')    
    label = row.has_cactus

    plt.subplot(4,4,i+1)
    plt.imshow(img)
    plt.text(0, -5, f'Class {label}', color='k')
        
    plt.axis('off')

plt.tight_layout()
plt.show()


# Load Model

In [ ]:
cnn = load_model('../input/models/Cactus/cactus_model_v01.h5')
cnn.summary()

# Sample Images

We will visualize the activations for two images. These are displayd below.

In [ ]:
row0 = train.iloc[6,:]
img0 = mpimg.imread(f'train/{row0.id}')    

row1 = train.iloc[4,:]
img1 = mpimg.imread(f'train/{row1.id}')    

plt.subplot(1,2,1)
plt.imshow(img0)
plt.text(0, -2, 'No Cactus', color='k')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(img1)
plt.text(0, -2, 'Cactus', color='k')
plt.axis('off')

plt.show()

In order to use the images in the functions below, we will need to reshape them into 4D arrays. We will also need to scale the pixel values.

In [ ]:
tensor0 = img0.reshape(1,32,32,3)/255
tensor1 = img1.reshape(1,32,32,3)/255

# Activation Visualization

In the cell below, we will create two functions for the purpose of visualizing the layer activations for an individual image. The `display_layer()` function is a helper function used to display a single layer of activations. The `display_activations()` function is the primary function. It will calculate and display the layer activations for each selected layer. 

In [ ]:
def display_layer(layer_index, activations, cmap):
    layer_activations = activations[layer_index]
    n_filters = layer_activations.shape[-1]
       
    n_cols = 8
    n_rows = n_filters // n_cols
    
    print(f'{cnn.layers[layer_index].name} - {n_filters} Filters')
    plt.figure(figsize=[2*n_cols, 2*n_rows])
    
    for i in range(n_filters):
        img = layer_activations[0,:,:,i]
        plt.subplot(n_rows, n_cols, i+1)
        plt.imshow(img, cmap=cmap)
        plt.axis('off')
    plt.show() 


def display_activations(img_tensor, layer_indices=[], cmap='viridis'):
    layer_outputs = [layer.output for layer in cnn.layers]
    activation_model = tf.keras.models.Model(inputs=cnn.inputs, outputs=layer_outputs)
    activations = activation_model(img_tensor)
    
    for i in layer_indices:
        display_layer(i, activations, cmap)

In [ ]:
display_activations(tensor1, [0,1,2,5,6,7], cmap='viridis')

In [ ]:
display_activations(tensor0, [0,1,2,5,6,7], cmap='viridis')